In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from polarDensity_helper import Coord_Get, get_header_info
from site_distributions import make_simple_site, combine_sites, plot_density

# Step 0: define a site

This is done with the plotting_show_binID notebook.
Some examples from ELIC:

In [ ]:
lipid = "DPPC"
leaf = "upp"
data_root = Path("/Users/ezry/Projects/ELIC_DPPC/elicPE17_DPPC")
fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

In [ ]:
num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
the_data = np.loadtxt(fpath, skiprows=1)
rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

In [ ]:
# onebin = make_simple_site(the_data, 25, 30, 1, Ntheta, dr, dth, exrho, frames, [29], f"M4 site, one bin")
# onebin_symmetric = make_simple_site(the_data, 25, 30, 1, Ntheta, dr, dth, exrho, frames, np.arange(5,30,6), "M4 site, one bin, symmetric sites")
# fourbins = make_simple_site(the_data, 25, 35, 2, Ntheta, dr, dth, exrho, frames, np.arange(28,30,1), "M4 site, four bins")

# symmetricA = make_simple_site(the_data, 25, 35, 1, Ntheta, dr, dth, exrho, frames, np.arange(5,30,6), "M4 site, four bins")
# symmetricB = make_simple_site(the_data, 25, 35, 1, Ntheta, dr, dth, exrho, frames, np.arange(4,30,6), "M4 site, four bins")
# fourbins_symmetric = combine_sites([symmetricA, symmetricB], exrho, "Four bins, symmetric")

extrabin = make_simple_site(the_data, 20, 25, 1, Ntheta, dr, dth, exrho, frames, [28])
fivebins = combine_sites([extrabin, fourbins], exrho, "Five bins")

# symmetricextra = make_simple_site(the_data, 20, 25, 1, Ntheta, dr, dth, exrho, frames, np.arange(4,30,6))
# fivebins_symmetric = combine_sites([symmetricA, symmetricB, symmetricextra], exrho, "Five bins, symmetric")

# Step 1 get the total area of the site in question

In [ ]:
the_area = fivebins.area

# Step 2 get the density 
In VMD:
1. load the trajectory of the empty membrane
2. modify do_get_counts.tcl to have the total area of the site
3. source do_get_counts.tcl
4. It will save to a file called counts_[area].out
5. Provide the path here:

In [ ]:
bulk_counts_path = Path("/Users/ezry/Projects/Polar_Binning_DeltaG/accessible_area/counts_154.out")
bulk_counts = np.loadtxt(bulk_counts_path)

frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(range(len(probabilities)), probabilities)
ax.set_ylabel("Probability")
ax.set_xlabel(f"Number of beads in an area about {int(the_area)} "+r"$\AA^2$")
bulk_mode = np.argmax(probabilities)
ax.vlines([bulk_mode], 0, np.max(probabilities), color = 'black', linestyles='dashed', label = f"mode={bulk_mode}")
ax.legend()

# Compare the bulk mode with the site mode

In [ ]:
fig, ax = plt.subplots(1,1)

# axes[0,0] = plot_density(onebin, axes[0,0])
# axes[0,1] = plot_density(onebin_symmetric, axes[0,1])
# axes[1,0] = plot_density(fourbins, axes[1,0])
# axes[1,1] = plot_density(fourbins_symmetric, axes[1,1])
ax = plot_density(fivebins, ax)
# axes[2,1] = plot_density(fivebins_symmetric, axes[2,1])



In [ ]:
expt_mode = np.argmax(np.bincount(fivebins.counts.astype(int).flatten()))
expt_mode

# The accessible area is...
$A_\mathrm{acc} = A_\mathrm{total} \frac{M_\mathrm{site}}{M_\mathrm{bulk}}$

In [ ]:
A_acc = the_area * expt_mode / bulk_mode
print(A_acc)